In [2]:
import pandas as pd
import requests
import zipfile
from datetime import datetime
from dateutil.relativedelta import relativedelta
import quandl

pd.options.display.float_format = '{:.4f}'.format

# Código completo na descrição

In [3]:
anos = 2 #por conta da fonte de dados, só funciona até começo de 2021
mes = []
ano = []

for i in range(anos * 12 - 1):
    mes.append((datetime.today()-relativedelta(months=i+1)).month)
    ano.append((datetime.today()-relativedelta(months=i+1)).year)

mes = mes[::-1]
ano = ano[::-1]

for i in range(len(mes)):
    if len(str(mes[i])) < 2:
        mes[i] = "0" + str(mes[i])
    else:
        mes[i] = str(mes[i])

for i in range(len(ano)):
    ano[i] = str(ano[i])

print(mes)
print(ano)

['05', '06', '07', '08', '09', '10', '11', '12', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '01', '02', '03']
['2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2023', '2023', '2023']


In [4]:
dados_fundos = pd.DataFrame()

for i in range(len(mes)):
    url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano[i]}{mes[i]}.zip'

    download = requests.get(url)

    with open(f"inf_diario_fi_{ano[i]}{mes[i]}.zip", "wb") as arquivo_cvm:

        arquivo_cvm.write(download.content) 
        
    arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano[i]}{mes[i]}.zip")

    dados_fundos_mes = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep = ";", encoding = 'ISO-8859-1')

    dados_fundos = dados_fundos.append(dados_fundos_mes)

dados_fundos.shape

    

C:\Users\rmnf\AppData\Local\Temp\ipykernel_10180\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_10180\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_10180\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_10180\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.appen

(11205962, 9)

In [5]:
dados_fundos.tail()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
568749,FI,97.929.213/0001-34,2023-03-27,79057838.4800,10.4395,79333122.7300,0.0000,0.0000,2
568750,FI,97.929.213/0001-34,2023-03-28,79160950.6700,10.4530,79435739.9800,0.0000,0.0000,2
568751,FI,97.929.213/0001-34,2023-03-29,79183843.5600,10.4560,79458137.4400,0.0000,0.0000,2
568752,FI,97.929.213/0001-34,2023-03-30,79365876.5900,10.4798,79639674.9200,0.0000,0.0000,2
568753,FI,97.929.213/0001-34,2023-03-31,79380059.6600,10.4816,79653361.5600,0.0000,0.0000,2


In [6]:
dados_cadastro = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', 
                             sep = ";", encoding = 'ISO-8859-1')

dados_cadastro = dados_cadastro[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

dados_cadastro = dados_cadastro.drop_duplicates()

dados_cadastro

C:\Users\rmnf\AppData\Local\Temp\ipykernel_10180\3548788005.py:1: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_cadastro = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',


,CNPJ_FUNDO,DENOM_SOCIAL
0,00.000.684/0001-21,DEUTSCHE BANK FDO APLIC QUOTAS FDO INV FINANCE...
1,00.000.731/0001-37,ITAMARITI CASH FUNDO APLICACAO QUOTAS FDOS INV...
2,00.000.732/0001-81,FUNDO APLIC. QUOTAS DE F.I. SANTANDER CURTO PRAZO
3,00.000.740/0001-28,FUNDO DE APLIC EM QUOTAS DE FUNDOS DE INV BMC ...
4,00.000.749/0001-39,BALANCE FUNDO APLICACAO QUOTAS FUNDO INVESTIME...
...,...,...
71944,28.850.140/0001-78,FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGR...
71945,28.912.043/0001-62,HAPVIDA - FUNDO DE FINANCIAMENTO DA INDUSTRIA ...
71946,31.674.280/0001-47,MEDIA VENTURE ESG INCENTIVADO - FUNDO DE FINAN...
71947,32.222.962/0001-81,Fundo de Financiamento da Indústria Cinematogr...


In [7]:
data_inicio = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[0]
data_fim = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[-1]
        

dados_fundos_filtrado = dados_fundos[(dados_fundos['DT_COMPTC'].isin([data_inicio, data_fim]))]       

dados_fundos_filtrado.tail()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
568661,FI,97.548.164/0001-90,2023-03-31,58763666.9000,0.1490,58645863.1900,0.0000,0.0000,1
568684,FI,97.548.167/0001-23,2023-03-31,5014910980.1600,4.4620,5033458186.7900,677100.0000,37344611.1900,12
568707,FI,97.711.801/0001-05,2023-03-31,86700600.8400,2.8983,86664862.3400,0.0000,0.0000,1
568730,FI,97.929.197/0001-80,2023-03-31,124368732.5000,4.5726,124252429.5700,0.0000,0.0000,13
568753,FI,97.929.213/0001-34,2023-03-31,79380059.6600,10.4816,79653361.5600,0.0000,0.0000,2


In [8]:
dados_fundos_filtrado.head()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2021-05-03,1078986.1700,27.4381,1078617.6800,0.0000,0.0000,1
21,FI,00.068.305/0001-35,2021-05-03,41640010.8600,27.4955,41623192.4800,0.0000,5051.3300,7013
42,FI,00.071.477/0001-68,2021-05-03,32777355454.2600,10.0406,32774945732.5900,2917789992.8400,2706550120.5500,296635
63,FI,00.073.041/0001-08,2021-05-03,17140830.8300,28.9167,17134440.4600,0.0000,0.0000,928
84,FI,00.083.181/0001-67,2021-05-03,17606182552.8000,763.1421,17784926612.3200,0.0000,0.0000,4


In [9]:
dados_fundos_filtrado.shape

(45391, 9)

In [10]:
base_final = pd.merge(dados_fundos_filtrado, dados_cadastro, how = "left", 
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])

base_final = base_final[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]

base_final

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST
0,00.017.024/0001-53,FUNDO DE INVESTIMENTO RENDA FIXA EXPONENCIAL,2021-05-03,27.4381,1078617.6800,1
1,00.068.305/0001-35,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2021-05-03,27.4955,41623192.4800,7013
2,00.071.477/0001-68,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,2021-05-03,10.0406,32774945732.5900,296635
3,00.073.041/0001-08,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,2021-05-03,28.9167,17134440.4600,928
4,00.083.181/0001-67,OPP I FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍVEL...,2021-05-03,763.1421,17784926612.3200,4
...,...,...,...,...,...,...
45390,97.548.164/0001-90,DYNAMIC II - FUNDO DE INVESTIMENTO MULTIMERCAD...,2023-03-31,0.1490,58645863.1900,1
45391,97.548.167/0001-23,JGP STRATEGY MASTER FUNDO DE INVESTIMENTO MULT...,2023-03-31,4.4620,5033458186.7900,12
45392,97.711.801/0001-05,953 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS D...,2023-03-31,2.8983,86664862.3400,1
45393,97.929.197/0001-80,LECT FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS ...,2023-03-31,4.5726,124252429.5700,13


In [11]:
fundos_prev = base_final[base_final['DENOM_SOCIAL'].str.contains("PREV", na = False)]
fundos_prev = fundos_prev[fundos_prev.VL_PATRIM_LIQ >= 10000000000]
fundos_prev = fundos_prev[fundos_prev.NR_COTST == 1]
fundos_prev = fundos_prev.sort_values(by="VL_PATRIM_LIQ", ascending=False)

fundos_prev

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST
2139,07.919.956/0001-30,BRASILPREV RT FIX VI FUNDO DE INVESTIMENTO EM ...,2021-05-03,3.5347,36035655050.3100,1
566,03.537.407/0001-40,BRASILPREV RT FIX II FUNDO DE INVESTIMENTO EM ...,2021-05-03,8.8687,34905627408.5900,1
22535,07.919.956/0001-30,BRASILPREV RT FIX VI FUNDO DE INVESTIMENTO EM ...,2023-03-31,4.1147,31755718944.0700,1
1611,06.001.785/0001-01,BRASILPREV RT FIX VII FUNDO DE INVESTIMENTO EM...,2021-05-03,3.2795,31000815364.6500,1
21071,03.537.407/0001-40,BRASILPREV RT FIX II FUNDO DE INVESTIMENTO EM ...,2023-03-31,10.2747,28786357337.3500,1
22055,06.001.785/0001-01,BRASILPREV RT FIX VII FUNDO DE INVESTIMENTO EM...,2023-03-31,3.8507,28332190166.3200,1
31472,29.259.706/0001-54,ITAÚ FLEXPREV HIGH YIELD RENDA FIXA CRÉDITO PR...,2023-03-31,13.3813,26274341930.0300,1
31473,29.259.723/0001-91,ITAÚ FLEXPREV HIGH YIELD RENDA FIXA CRÉDITO PR...,2023-03-31,13.7456,26223344634.6100,1
1208,05.061.121/0001-67,BRASILPREV RT FIX C FUNDO DE INVESTIMENTO EM C...,2021-05-03,6.1153,17107358620.0800,1
17229,36.347.890/0001-50,CARTEIRA ITAÚ ASSET DE PREVIDÊNCIA FICFI MULTI...,2021-05-03,11.0914,16150790679.0200,1


# Patrimônio e performance do fundo em dezembro

In [12]:
maior_fundo = fundos_prev[(fundos_prev['CNPJ_FUNDO'] == fundos_prev["CNPJ_FUNDO"].iloc[0])]

maior_fundo = maior_fundo.sort_values(by="DT_COMPTC")

patrimonio_do_fundo =  "R$ " + str((maior_fundo['VL_PATRIM_LIQ'].iloc[-1]/1000000000).round(2)) + " Bi"

patrimonio_do_fundo

'R$ 31.76 Bi'

In [13]:
                                       
retorno = (maior_fundo['VL_QUOTA'].iloc[-1]/maior_fundo['VL_QUOTA'].iloc[0] - 1)* 100

retorno.round(2)

16.41

In [14]:
quandl.ApiConfig.api_key = "YUSDTrHyCjE7Gd8dP_yn"

ipca_mes = quandl.get("BCB/433", start_date = f"{ano[0]}-{mes[0]}-01")
cdi_mes = quandl.get("BCB/4391", start_date = f"{ano[0]}-{mes[0]}-01")

ipca_mes

,Value
Date,
2021-05-31,0.8300
2021-06-30,0.5300
2021-07-31,0.9600
2021-08-31,0.8700
2021-09-30,1.1600
2021-10-31,1.2500
2021-11-30,0.9500
2021-12-31,0.7300
2022-01-31,0.5400


In [15]:
cdi_mes['retorno_acumulado'] = (1 + cdi_mes['Value']/100).cumprod() - 1
ipca_mes['retorno_acumulado'] = (1 + ipca_mes['Value']/100).cumprod() - 1

cdi_mes

,Value,retorno_acumulado
Date,,
2021-05-31,0.2700,0.0027
2021-06-30,0.3100,0.0058
2021-07-31,0.3600,0.0094
2021-08-31,0.4300,0.0138
2021-09-30,0.4400,0.0182
2021-10-31,0.4900,0.0232
2021-11-30,0.5900,0.0293
2021-12-31,0.7700,0.0372
2022-01-31,0.7300,0.0448


In [16]:
retorno_cdi = ((cdi_mes['retorno_acumulado'].iloc[-1]) * 100)

retorno_ipca = ((ipca_mes['retorno_acumulado'].iloc[-1]) * 100)

print("Retorno fundo: " + str(retorno.round(2)) + " %")
print("Retorno CDI: " + str(retorno_cdi.round(2)) + " %")
print("Retorno IPCA: " + str(retorno_ipca.round(2)) + " %" + "\n")
print("Retorno %CDI: " + str((retorno/retorno_cdi * 100).round(2)) + " %")
print("Retorno IPCA: IPCA + " + str((retorno - retorno_ipca).round(2)) + " %")

Retorno fundo: 16.41 %
Retorno CDI: 20.83 %
Retorno IPCA: 16.12 %

Retorno %CDI: 78.76 %
Retorno IPCA: IPCA + 0.29 %


Maior fundo previdenciário em patrimônio, não ganha para o CDI e não bate uma renda fixa pós em inflação mesmo correndo mais risco, é bom escolher bem o fundo previdenciário

# Melhorias

In [17]:
#1. Quais são os 5 maiores fundos de previdência privada do Brasil?

print("Os 5 maiores fundos de previdência privada do Brasil são:")
for i in range(5):
    print(fundos_prev['DENOM_SOCIAL'].iloc[i])

Os 5 maiores fundos de previdência privada do Brasil são:
BRASILPREV RT FIX VI FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA
BRASILPREV RT FIX II FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA
BRASILPREV RT FIX VI FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA
BRASILPREV RT FIX VII FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA
BRASILPREV RT FIX II FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA
